## 2D to 3D conversion with single Image

https://stackoverflow.com/questions/11334012/reconstruct-3d-coordinates-in-camera-coordinate-system-from-2d-pixels-with-sid
Assumptions: 3D points are on the Z=0 plane
Depth and scale information is lost when 3D points are projected onto a 2D image hence inverese problem 2D to 3D from single image is not possible.


In [85]:


import numpy as np
from numpy.linalg import inv, pinv
import cv2
import sys
import math


# Caamera-Calibration-Matrix

Cam_intrisic = [[1.017215234570303e+03, 0, 3.195000000000000e+02, 0], 
                 [  0, 1.012549014668498e+03,2.395000000000000e+02, 0],
                 [  0, 0, 1.0000, 0],
                 [0, 0, 0, 0]]
K=np.asanyarray(Cam_intrisic)
 

#Transforma Matrix from 3D-World-Coordinate System to 3D-Camera-Coordinate System (Origin on CCD-Chip) 
#% 
# The Camera is oriented "looking" into positive X-Direction of the World-Coordinate-System. On the picture,
# positive Y-Direction will be to the left, positive Z-Direction to the top. (right hand coordinate system!)
Extrinsic = [[-0.0113242625465167,   -0.999822053685344,   0.0151163536128891 ,  141.173585444427], 
                   [0.00842007509644635,   -0.0152123858102325,   -0.999848810645587,   1611.96528372161], 
                   [0.999900032304804,   -0.0111955728474261,   0.00859117128537919,  847.090629282911], 
                   [0,   0,   0,   1]] 
R_World_to_Cam=np.asanyarray(Extrinsic)
# Projection- and Transforma Matrix P 
P =np.matmul(K , R_World_to_Cam) 

# arbitrary Points X_World in World-Coordinate-System [mm] (homogenous Coordinates) 
# forming a square of size 10m x 4m 
X_World_1 =np.asanyarray([[20000], [2000], [0], [1]] ) 
X_World_2 =np.asanyarray([[20000], [-2000], [0], [1]] )
X_World_3 =np.asanyarray([[10000], [2000], [0], [1]] )
X_World_4 =np.asanyarray([[10000], [-2000], [0], [1]])
print("X_World_1=",X_World_1.shape)

#Transform and Project from 3D-World -> 2D-Picture 
X_Pic_1 = np.matmul(P, X_World_1) 
X_Pic_2 = np.matmul(P, X_World_2)
X_Pic_3 = np.matmul(P, X_World_3)
X_Pic_4 = np.matmul(P, X_World_4)
print ("\n X_World_1",X_World_1)
print("\n X_Pic_1", X_Pic_1)
print ("\n X_World_2",X_World_2)
print("\n X_Pic_2", X_Pic_2)
print ("\n X_World_3",X_World_3)
print("\n X_Pic_3", X_Pic_3)
print ("\n X_World_4",X_World_4)
print("\n X_Pic_4", X_Pic_4)

#'''
#normalize homogenous Coordinates (3rd Element has to be 1!) 
X_Pic_1 = X_Pic_1 / X_Pic_1[2,0]
X_Pic_2 = X_Pic_2 / X_Pic_2[2,0]
X_Pic_3 = X_Pic_3 / X_Pic_3[2,0] 
X_Pic_4 = X_Pic_4 / X_Pic_4[2,0]

print("After normalizing \n ")


print ("\n X_World_1",X_World_1)
print("\n X_Pic_1", X_Pic_1)
print ("\n X_World_2",X_World_2)
print("\n X_Pic_2", X_Pic_2)
print ("\n X_World_3",X_World_3)
print("\n X_Pic_3", X_Pic_3)
print ("\n X_World_4",X_World_4)
print("\n X_Pic_4", X_Pic_4)
#Now for reverse procedure take arbitrary points in Camera-Picture... 
#(for simplicity, take points from above and "go" 30px to the right and 40px down) 


Pix=np.asanyarray([[30], [40], [0]])
print("Pix",Pix.shape)
#print("X_Pic_1",X_Pic_1.type)
X_Pic_backtransform_1 = np.add(X_Pic_1[0:3,0].reshape(3,1),Pix)
X_Pic_backtransform_2 = np.add(X_Pic_2[0:3,0].reshape(3,1),Pix)
X_Pic_backtransform_3 = np.add(X_Pic_3[0:3,0].reshape(3,1),Pix)
X_Pic_backtransform_4 = np.add(X_Pic_4[0:3,0].reshape(3,1),Pix)




#print("X_Pic_backtransform_1",X_Pic_backtransform_1.shape)

# ... and transform back following the formula from the Master Thesis (in German):
# Ilker Savas, "Entwicklung eines Systems zur visuellen Positionsbestimmung von Interaktionspartnern" 
M_Mat = P[0:3,0:3]                 #% Matrix M is the "top-front" 3x3 part 

p_4 = P[0:3,3].reshape(3,1)                     #% Vector p_4 is the "top-rear" 1x3 part
print("p_4 shape",p_4.shape)
C_tilde = - np.matmul(inv( M_Mat ), p_4)    # % calculate C_tilde 

#% Invert Projection with Side-Condition ( Z = 0 ) and Transform back to 
#% World-Coordinate-System 
print("M_mat",M_Mat)
print("mat shape",M_Mat.shape)
print("X_Pic_backtransform_1 shape",X_Pic_backtransform_1.shape)

X_Tilde_1 =np.matmul( inv( M_Mat ) , X_Pic_backtransform_1 )
X_Tilde_2 = np.matmul( inv( M_Mat ) , X_Pic_backtransform_2 )
X_Tilde_3 = np.matmul( inv( M_Mat ) , X_Pic_backtransform_3 )
X_Tilde_4 = np.matmul( inv( M_Mat ) , X_Pic_backtransform_4 )

mue_N_1 = -C_tilde[2,0] / X_Tilde_1[2,0]
mue_N_2 = -C_tilde[2,0] / X_Tilde_2[2,0] 
mue_N_3 = -C_tilde[2,0] / X_Tilde_3[2,0] 
mue_N_4 = -C_tilde[2,0] / X_Tilde_4[2,0] 

#% Do the inversion of above steps... 
X_World_backtransform_1 = mue_N_1 * np.matmul(inv( M_Mat ) , X_Pic_backtransform_1 ) + C_tilde; 
X_World_backtransform_2 = mue_N_2 * np.matmul(inv( M_Mat ) , X_Pic_backtransform_2 ) + C_tilde; 
X_World_backtransform_3 = mue_N_3 * np.matmul(inv( M_Mat ) , X_Pic_backtransform_3 ) + C_tilde; 
X_World_backtransform_4 = mue_N_4 * np.matmul(inv( M_Mat ) , X_Pic_backtransform_4 ) + C_tilde; 

print("After reverse transform \n ")



print("\n X_Pic_1", X_Pic_1)
print ("\n X_Pic_backtransform_1",X_World_backtransform_1)

print("\n X_Pic_2", X_Pic_2)
print ("\n X_Pic_backtransform_2",X_World_backtransform_2)

print("\n X_Pic_3", X_Pic_3)
print ("\n X_Pic_backtransform_3",X_World_backtransform_3)

print("\n X_Pic_4", X_Pic_4)
print ("\n X_Pic_backtransform_4",X_World_backtransform_4)




X_World_1= (4, 1)

 X_World_1 [[20000]
 [ 2000]
 [    0]
 [    1]]

 X_Pic_1 [[4532003.91587905]
 [6758938.74309251]
 [  20822.70012968]
 [      0.        ]]

 X_World_2 [[20000]
 [-2000]
 [    0]
 [    1]]

 X_Pic_2 [[8614448.75745046]
 [6831277.24693197]
 [  20867.48242107]
 [      0.        ]]

 X_World_3 [[10000]
 [ 2000]
 [    0]
 [    1]]

 X_Pic_3 [[1452515.43649111]
 [4278920.77829909]
 [  10823.69980664]
 [      0.        ]]

 X_World_4 [[10000]
 [-2000]
 [    0]
 [    1]]

 X_Pic_4 [[5534960.27806252]
 [4351259.28213855]
 [  10868.48209803]
 [      0.        ]]
After normalizing 
 

 X_World_1 [[20000]
 [ 2000]
 [    0]
 [    1]]

 X_Pic_1 [[217.64727378]
 [324.5947308 ]
 [  1.        ]
 [  0.        ]]

 X_World_2 [[20000]
 [-2000]
 [    0]
 [    1]]

 X_Pic_2 [[412.81686902]
 [327.36470596]
 [  1.        ]
 [  0.        ]]

 X_World_3 [[10000]
 [ 2000]
 [    0]
 [    1]]

 X_Pic_3 [[134.19768309]
 [395.32884824]
 [  1.        ]
 [  0.        ]]

 X_World_4 [[10000]
 [-2000]